# 1. Install packages

In [1]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append("../input/segmentation-models-pytorch/segmentation_models.pytorch-0.2.1")
sys.path.append("../input/pretrainedmodels/pretrainedmodels-0.7.4")
sys.path.append("../input/efficientnet-pytorch/EfficientNet-PyTorch-master")

!pip install ../input/mmdetection/addict-2.4.0-py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/yapf-0.31.0-py2.py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/terminaltables-3.1.0-py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/einops* > /dev/null
!pip install ../input/mmdetection/mmcv_full-1.3.17-cp37-cp37m-linux_x86_64.whl > /dev/null

# 2. Install mmsegmentation 

This is from my own [mmseg github repo](https://github.com/CarnoZhao/Kaggle-UWMGIT) (leave a star if you like it!)

I have integrated `segmentation_models_pytorch` in this version of `mmsegmentation`. Although `segmentation_models_pytorch`'s simple Unet performs better than some models of `mmsegmentation`, anyway, `mmsegmentation` is still a good library for segmentation task when you want to compare various models in a unified training pipeline.

I only hard-coded `smp.Unet` in `./mmseg/models/segmentors/smp_models.py`. You can add more `smp` models in it!

In [2]:
!git clone https://github.com/CarnoZhao/Kaggle-UWMGIT && cd Kaggle-UWMGIT && pip install -e .

Cloning into 'Kaggle-UWMGIT'...
remote: Enumerating objects: 5361, done.
remote: Counting objects: 100% (5361/5361), done.
remote: Compressing objects: 100% (1126/1126), done.
remote: Total 5361 (delta 4185), reused 5361 (delta 4185), pack-reused 0
Receiving objects: 100% (5361/5361), 5.03 MiB | 8.24 MiB/s, done.
Resolving deltas: 100% (4185/4185), done.
Obtaining file:///kaggle/working/Kaggle-UWMGIT
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation


# 3. Prepare data

In [3]:
import os
import glob
import numpy as np
import pandas as pd

import cv2
from PIL import Image
from tqdm.auto import tqdm

## 3.1 Read csv and extract meta info

In [4]:
df_train = pd.read_csv("../input/uw-madison-gi-tract-image-segmentation/train.csv")
df_train = df_train.sort_values(["id", "class"]).reset_index(drop = True)
df_train["patient"] = df_train.id.apply(lambda x: x.split("_")[0])
df_train["days"] = df_train.id.apply(lambda x: "_".join(x.split("_")[:2]))

all_image_files = sorted(glob.glob("../input/uw-madison-gi-tract-image-segmentation/train/*/*/scans/*.png"), key = lambda x: x.split("/")[3] + "_" + x.split("/")[5])
size_x = [int(os.path.basename(_)[:-4].split("_")[-4]) for _ in all_image_files]
size_y = [int(os.path.basename(_)[:-4].split("_")[-3]) for _ in all_image_files]
spacing_x = [float(os.path.basename(_)[:-4].split("_")[-2]) for _ in all_image_files]
spacing_y = [float(os.path.basename(_)[:-4].split("_")[-1]) for _ in all_image_files]
df_train["image_files"] = np.repeat(all_image_files, 3)
df_train["spacing_x"] = np.repeat(spacing_x, 3)
df_train["spacing_y"] = np.repeat(spacing_y, 3)
df_train["size_x"] = np.repeat(size_x, 3)
df_train["size_y"] = np.repeat(size_y, 3)
df_train["slice"] = np.repeat([int(os.path.basename(_)[:-4].split("_")[-5]) for _ in all_image_files], 3)
df_train

,id,class,segmentation,patient,days,image_files,spacing_x,spacing_y,size_x,size_y,slice
0,case101_day20_slice_0001,large_bowel,NaN,case101,case101_day20,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,266,266,6
1,case101_day20_slice_0001,small_bowel,NaN,case101,case101_day20,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,266,266,6
2,case101_day20_slice_0001,stomach,NaN,case101,case101_day20,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,266,266,6
3,case101_day20_slice_0002,large_bowel,NaN,case101,case101_day20,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,266,266,82
4,case101_day20_slice_0002,small_bowel,NaN,case101,case101_day20,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,266,266,82
...,...,...,...,...,...,...,...,...,...,...,...
115483,case9_day22_slice_0143,small_bowel,NaN,case9,case9_day22,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,360,310,93
115484,case9_day22_slice_0143,stomach,NaN,case9,case9_day22,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,360,310,93
115485,case9_day22_slice_0144,large_bowel,NaN,case9,case9_day22,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,360,310,135
115486,case9_day22_slice_0144,small_bowel,NaN,case9,case9_day22,../input/uw-madison-gi-tract-image-segmentatio...,1.5,1.5,360,310,135


## 3.2 Make mmseg-format data (2.5D by default)


Here, I used 2.5d data with stride=2. Thanks this good trick from [https://www.kaggle.com/code/awsaf49/uwmgi-2-5d-stride-2-data](https://www.kaggle.com/code/awsaf49/uwmgi-2-5d-stride-2-data)

In [9]:
def rle_decode(mask_rle, shape):
    s = np.array(mask_rle.split(), dtype=int)
    starts, lengths = s[0::2] - 1, s[1::2]
    ends = starts + lengths
    h, w = shape
    img = np.zeros((h * w,), dtype = np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape)

!mkdir -p ./mmseg_train/{images,labels,splits}
for day, group in tqdm(df_train.groupby("days")):
    patient = group.patient.iloc[0]
    imgs = []
    msks = []
    file_names = []
    for file_name in group.image_files.unique():
        img = cv2.imread(file_name, cv2.IMREAD_ANYDEPTH)
        segms = group.loc[group.image_files == file_name]
        masks = {}
        for segm, label in zip(segms.segmentation, segms["class"]):
            if not pd.isna(segm):
                mask = rle_decode(segm, img.shape[:2])
                masks[label] = mask
            else:
                masks[label] = np.zeros(img.shape[:2], dtype = np.uint8)
        masks = np.stack([masks[k] for k in sorted(masks)], -1)
        imgs.append(img)
        msks.append(masks)
        
    imgs = np.stack(imgs, 0)
    msks = np.stack(msks, 0)
    for i in range(msks.shape[0]):
        img = imgs[[max(0, i - 2), i, min(imgs.shape[0] - 1, i + 2)]].transpose(1,2,0) # 2.5d data
        msk = msks[i]
        new_file_name = f"{day}_{i}.png"
        cv2.imwrite(f"./mmseg_train/images/{new_file_name}", img)
        cv2.imwrite(f"./mmseg_train/labels/{new_file_name}", msk)

  0%|          | 0/274 [00:00<?, ?it/s]

## 3.3 Make fold splits

In [10]:
all_image_files = glob.glob("./mmseg_train/images/*")
patients = [os.path.basename(_).split("_")[0] for _ in all_image_files]


from sklearn.model_selection import GroupKFold

split = list(GroupKFold(5).split(patients, groups = patients))

for fold, (train_idx, valid_idx) in enumerate(split):
    with open(f"./mmseg_train/splits/fold_{fold}.txt", "w") as f:
        for idx in train_idx:
            f.write(os.path.basename(all_image_files[idx])[:-4] + "\n")
    with open(f"./mmseg_train/splits/holdout_{fold}.txt", "w") as f:
        for idx in valid_idx:
            f.write(os.path.basename(all_image_files[idx])[:-4] + "\n")

# 4. Training

## 4.1 Make config

This is only **a simple baseline**, you can change anything in it

From my own experiment, when using larger backbone, larger image size and more augs, the public score will be easily exceed 0.865.

Here, I only train for 1k iters. **More iters are required to get a valid score**.

I have made a single model submission scored 0.878 using this training pipeline!

In [33]:
%%bash

cat <<EOT >> ./Kaggle-UWMGIT/config.py
num_classes = 3

# model settings
norm_cfg = dict(type='SyncBN', requires_grad=True)
loss = [
    dict(type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
]
model = dict(
    type='SMPUnet',
    backbone=dict(
        type='timm-efficientnet-b0',
        pretrained="imagenet"
    ),
    decode_head=dict(
        num_classes=num_classes,
        align_corners=False,
        loss_decode=loss
    ),
    # model training and testing settings
    train_cfg=dict(),
    test_cfg=dict(mode="whole", multi_label=True))

# dataset settings
dataset_type = 'CustomDataset'
data_root = '../mmseg_train/'
classes = ['large_bowel', 'small_bowel', 'stomach']
palette = [[0,0,0], [128,128,128], [255,255,255]]
img_norm_cfg = dict(mean=[0,0,0], std=[1,1,1], to_rgb=True)
size = 256
albu_train_transforms = [
    dict(type='RandomBrightnessContrast', p=0.5),
]
train_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True, color_type='unchanged', max_value='max'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(size, size), keep_ratio=True),
    dict(type='RandomFlip', prob=0.5, direction='horizontal'),
    dict(type='Albu', transforms=albu_train_transforms),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size=(size, size), pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
test_pipeline = [
    dict(type='LoadImageFromFile', to_float32=True, color_type='unchanged', max_value='max'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(size, size),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size=(size, size), pad_val=0, seg_pad_val=255),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=8,
    workers_per_gpu=4,
    train=dict(
        type=dataset_type,
        multi_label=True,
        data_root=data_root,
        img_dir='images',
        ann_dir='labels',
        img_suffix=".png",
        seg_map_suffix='.png',
        split="splits/fold_0.txt",
        classes=classes,
        palette=palette,
        pipeline=train_pipeline),
    val=dict(
        type=dataset_type,
        multi_label=True,
        data_root=data_root,
        img_dir='images',
        ann_dir='labels',
        img_suffix=".png",
        seg_map_suffix='.png',
        split="splits/holdout_0.txt",
        classes=classes,
        palette=palette,
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        multi_label=True,
        data_root=data_root,
        test_mode=True,
        img_dir='test/images',
        ann_dir='test/labels',
        img_suffix=".jpg",
        seg_map_suffix='.png',
        classes=classes,
        palette=palette,
        pipeline=test_pipeline))

# yapf:disable
log_config = dict(
    interval=50,
    hooks=[
        dict(type='CustomizedTextLoggerHook', by_epoch=False),
    ])
# yapf:enable
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]
cudnn_benchmark = True

total_iters = 1
# optimizer
optimizer = dict(type='AdamW', lr=1e-3, betas=(0.9, 0.999), weight_decay=0.05)
optimizer_config = dict(type='Fp16OptimizerHook', loss_scale='dynamic')
# learning policy
lr_config = dict(policy='poly',
                 warmup='linear',
                 warmup_iters=500,
                 warmup_ratio=1e-6,
                 power=1.0, min_lr=0.0, by_epoch=False)
# runtime settings
find_unused_parameters=True
runner = dict(type='IterBasedRunner', max_iters=int(total_iters * 1000))
checkpoint_config = dict(by_epoch=False, interval=int(total_iters * 1000), save_optimizer=False)
evaluation = dict(by_epoch=False, interval=min(5000, int(total_iters * 1000)), metric=['imDice', 'mDice'], pre_eval=True)
fp16 = dict()

work_dir = f'./work_dirs/tract/baseline'
EOT

# 4.2 Let's start training

In [23]:
# reinstall for inner bash usage
!cp -r ../input/segmentation-models-pytorch/segmentation_models.pytorch-0.2.1 ./ && cd segmentation_models.pytorch-0.2.1  && pip install -e .
!cp -r ../input/timm-pytorch-image-models/pytorch-image-models-master ./ && cd pytorch-image-models-master  && pip install -e .

Obtaining file:///kaggle/working/segmentation_models.pytorch-0.2.1
  Preparing metadata (setup.py) ... done
  Using cached pretrainedmodels-0.7.4-py3-none-any.whl
  Using cached timm-0.4.12-py3-none-any.whl (376 kB)
  Attempting uninstall: timm
    Found existing installation: timm 0.5.4
    Uninstalling timm-0.5.4:
      Successfully uninstalled timm-0.5.4
  Running setup.py develop for segmentation-models-pytorch
Obtaining file:///kaggle/working/pytorch-image-models-master
  Preparing metadata (setup.py) ... done
  Attempting uninstall: timm
    Found existing installation: timm 0.4.12
    Uninstalling timm-0.4.12:
      Successfully uninstalled timm-0.4.12
  Running setup.py develop for timm
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
segmentation-models-pytorch 0.2.1 requires timm==0.4.12, but you have timm 0.5.4 which is incompatible.


In [38]:
%%bash

cd Kaggle-UWMGIT

python ./tools/train.py ./config.py --gpu-ids 0

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 7664/7664, 21.7 task/s, elapsed: 354s, ETA:     0s

# 5. Inferencing

## 5.1 Load trained models

In [45]:
sys.path.append('./Kaggle-UWMGIT')
from mmseg.apis import init_segmentor, inference_segmentor
from mmcv.utils import config

cfgs = [
    "./Kaggle-UWMGIT/work_dirs/tract/baseline/config.py",
]

ckpts = [
    "./Kaggle-UWMGIT/work_dirs/tract/baseline/latest.pth",
]

models = []
for cfg, ckpt in zip(cfgs, ckpts):
    cfg = config.Config.fromfile(cfg)
    cfg.model.backbone.pretrained = None
    cfg.model.test_cfg.logits = True
    cfg.data.test.pipeline[1].transforms.insert(2, dict(type="Normalize", mean=[0,0,0], std=[1,1,1], to_rgb=False))

    model = init_segmentor(cfg, ckpt)
    models.append(model)

load checkpoint from local path: ./Kaggle-UWMGIT/work_dirs/tract/baseline/latest.pth


## 5.2 Make test submission csv

In [46]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
from scipy.ndimage import binary_closing, binary_opening, measurements

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

classes = ['large_bowel', 'small_bowel', 'stomach']
data_dir = "../input/uw-madison-gi-tract-image-segmentation/"
test_dir = os.path.join(data_dir, "test")
sub = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
test_images = glob.glob(os.path.join(test_dir, "**", "*.png"), recursive = True)

if len(test_images) == 0:
    test_dir = os.path.join(data_dir, "train")
    sub = pd.read_csv(os.path.join(data_dir, "train.csv"))[["id", "class"]].iloc[:100 * 3]
    sub["predicted"] = ""
    test_images = glob.glob(os.path.join(test_dir, "**", "*.png"), recursive = True)
    
id2img = {_.rsplit("/", 4)[2] + "_" + "_".join(_.rsplit("/", 4)[4].split("_")[:2]): _ for _ in test_images}
sub["file_name"] = sub.id.map(id2img)
sub["days"] = sub.id.apply(lambda x: "_".join(x.split("_")[:2]))
fname2index = {f + c: i for f, c, i in zip(sub.file_name, sub["class"], sub.index)}
sub

,id,class,predicted,file_name,days
0,case123_day20_slice_0001,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
1,case123_day20_slice_0001,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
2,case123_day20_slice_0001,stomach,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
3,case123_day20_slice_0002,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
4,case123_day20_slice_0002,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
...,...,...,...,...,...
295,case123_day20_slice_0099,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
296,case123_day20_slice_0099,stomach,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
297,case123_day20_slice_0100,large_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20
298,case123_day20_slice_0100,small_bowel,,../input/uw-madison-gi-tract-image-segmentatio...,case123_day20


## 5.3 Start Inferencing

In [53]:
subs = []
for day, group in tqdm(sub.groupby("days")):
    imgs = []
    for file_name in group.file_name.unique():
        img = cv2.imread(file_name, cv2.IMREAD_ANYDEPTH)
        old_size = img.shape[:2]
        s = int(os.path.basename(file_name).split("_")[1])
        file_names = [file_name.replace(f"slice_{s:04d}", f"slice_{s + i:04d}") for i in range(-2, 3)]
        file_names = [_ for _ in file_names if os.path.exists(_)]
        imgs = [cv2.imread(file_names[0], cv2.IMREAD_ANYDEPTH)] + [img] + [cv2.imread(file_names[-1], cv2.IMREAD_ANYDEPTH)]
        
        new_img = np.stack(imgs, -1)
        new_img = new_img.astype(np.float32) / new_img.max()

        res = [inference_segmentor(model, new_img)[0] for model in models]
        res = (sum(res) / len(res)).round().astype(np.uint8)
        res = cv2.resize(res, old_size[::-1], interpolation = cv2.INTER_NEAREST)
        for j in range(3):
            rle = rle_encode(res[...,j])
            index = fname2index[file_name + classes[j]]
            sub.loc[index, "predicted"] = rle

  0%|          | 0/1 [00:00<?, ?it/s]

## 5.4 Format submission

In [54]:
sub = sub[["id", "class", "predicted"]]
sub.to_csv("submission.csv", index = False)
sub

,id,class,predicted
0,case123_day20_slice_0001,large_bowel,
1,case123_day20_slice_0001,small_bowel,
2,case123_day20_slice_0001,stomach,
3,case123_day20_slice_0002,large_bowel,
4,case123_day20_slice_0002,small_bowel,
...,...,...,...
295,case123_day20_slice_0099,small_bowel,
296,case123_day20_slice_0099,stomach,
297,case123_day20_slice_0100,large_bowel,
298,case123_day20_slice_0100,small_bowel,
